In [9]:
import gc
import time
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from itertools import combinations, groupby
from collections import Counter

# Ignore all warnings
import warnings
warnings.filterwarnings('ignore')

# Check all avaliable files
from subprocess import check_output
print(check_output(["ls", "./"]).decode("utf8"))

3d-scatter-tupac-with-mac.html
AskWhai Data Challenge.ipynb
Association Rule Analysis.ipynb
D3 Tree Map
EDA-Insights.ipynb
Feature Engineering and Modeling.ipynb
aisles.csv
da_count.csv
da_order.csv
dataset
depart_aisle_count.csv
depart_aisle_order.csv
departments.csv
mydf.csv
order_products__prior.csv
order_products__test_cap.csv
order_products__train_cap.csv
orders.csv
products.csv
products_new.csv
summary.csv
train_data.csv
treemap_order.png
treemap_products.png
user_all.csv
users.csv



In [2]:
# Read in the data set
order_products_prior = pd.read_csv('./order_products__prior.csv')

In [3]:
order_products_prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [4]:
order_products_prior.dtypes

order_id             int64
product_id           int64
add_to_cart_order    int64
reordered            int64
dtype: object

In [ ]:
# import pickle
# with open('dataset', 'wb') as f:
#     pickle.dump(dataset, f)
    
# import pickle
# with open('dataset', 'rb') as f:
#     dataset = pickle.load(f)

In [5]:
orders = order_products_prior.set_index('order_id')['product_id'].rename('item_id')

In [6]:
# Returns frequency counts for items or item pairs
def count_freq(iterable):
     '''
     If iterable is a serise of items, them use regular value_count:
     If it is a list of paired items, then use Counter()
     
     Example of Counter:
     Counter({('apple', 'egg'): 1, ('apple', 'milk'): 1, ('egg', 'milk'): 2})
     Output: Counter({('apple', 'egg'): 1, ('apple', 'milk'): 1, ('egg', 'milk'): 2})
    
     '''
    
    if type(iterable) == pd.core.series.Series:
        return iterable.value_counts().rename("freq")
    else: 
        return pd.Series(Counter(iterable)).rename("freq")
    
# Returns number of unique orders
def unique_order_count(order):
    return len(set(order.index))

In [7]:
# Calculate single items' frequency:
item_stats = count_freq(orders).to_frame("freq")
item_stats['support'] = item_stats['freq'] / unique_order_count(orders) * 100

# Remove the items that has frequency below min support 
qualifying_items= item_stats[item_stats['support'] >= 0.02].index
orders  = orders[orders.isin(qualifying_items)]

In [12]:
# Remove the orders with only one item
order_size = count_freq(orders.index)
qualifying_orders = order_size[order_size >= 2].index
orders = orders[orders.index.isin(qualifying_orders)]

print("Remaining orders with 2+ items: {:11d}".format(len(qualifying_orders)))
print("Remaining order_item: {:21d}".format(len(orders)))

Remaining orders with 2+ items:     2975061
Remaining order_item:              27848926


In [15]:
# re-calculated the frequency of items after removed low-frequency items
item_stats = count_freq(orders).to_frame("freq")
item_stats['support'] = item_stats['freq'] / unique_order_count(orders) * 100

In [16]:
# A function to generate item pairs
def get_item_pairs(order_item):
    order_item = order_item.reset_index().as_matrix()
    for order_id, order_object in groupby(order_item, lambda x: x[0]):
        item_list = [item[1] for item in order_object]
        
        for item_pair in combinations(item_list, 2):
            yield item_pair

In [19]:
# Get item pairs
item_pairs = get_item_pairs(orders)

# Calculate item pair frequency and support
item_pairs_fre = count_freq(item_pairs).to_frame("freqAB")
item_pairs_fre['supportAB'] = item_pairs_fre['freqAB'] / len(qualifying_orders) * 100

Item pairs:                        20384055


In [27]:
# Create table of association rules and compute relevant metrics
item_pairs_fre = item_pairs_fre.reset_index().rename(columns={'level_0': 'item_A', 'level_1': 'item_B'})

In [28]:
item_pairs_fre

,item_A,item_B,freqAB,supportAB
0,33120,28985,437,0.014689
1,33120,9327,39,0.001311
2,33120,45918,7,0.000235
3,33120,17794,382,0.012840
4,33120,40141,6,0.000202
...,...,...,...,...
20384050,32700,12023,1,0.000034
20384051,12023,47941,1,0.000034
20384052,7854,39678,1,0.000034
20384053,45309,39678,1,0.000034


In [29]:
item_pairs_fre = item_pairs_fre.merge(item_stats.rename(columns={'freq': 'freqA', 'support': 'supportA'}), left_on='item_A', right_index=True).merge(item_stats.rename(columns={'freq': 'freqB', 'support': 'supportB'}), left_on='item_B', right_index=True)

In [30]:
item_pairs_fre

,item_A,item_B,freqAB,supportAB,freqA,supportA,freqB,supportB
0,33120,28985,437,0.014689,19294,0.648525,67298,2.262071
258232,9327,28985,74,0.002487,6264,0.210550,67298,2.262071
3487193,45918,28985,19,0.000639,737,0.024773,67298,2.262071
1361,17794,28985,2229,0.074923,72583,2.439715,67298,2.262071
1619646,40141,28985,12,0.000403,1177,0.039562,67298,2.262071
...,...,...,...,...,...,...,...,...
9738241,16221,43556,2,0.000067,828,0.027831,941,0.031630
8990254,17819,43556,1,0.000034,1042,0.035024,941,0.031630
5579795,41651,43556,39,0.001311,933,0.031361,941,0.031630
11446859,1154,43556,1,0.000034,674,0.022655,941,0.031630


In [32]:
item_pairs_fre['confidenceAtoB'] = item_pairs_fre['supportAB'] / item_pairs_fre['supportA']
item_pairs_fre['confidenceBtoA'] = item_pairs_fre['supportAB'] / item_pairs_fre['supportB']
item_pairs_fre['lift'] = item_pairs_fre['supportAB'] / (item_pairs_fre['supportA'] * item_pairs_fre['supportB'])

In [33]:
item_pairs_fre.head()

,item_A,item_B,freqAB,supportAB,freqA,supportA,freqB,supportB,confidenceAtoB,confidenceBtoA,lift
0,33120,28985,437,0.014689,19294,0.648525,67298,2.262071,0.022650,0.006494,0.010013
258232,9327,28985,74,0.002487,6264,0.210550,67298,2.262071,0.011814,0.001100,0.005222
3487193,45918,28985,19,0.000639,737,0.024773,67298,2.262071,0.025780,0.000282,0.011397
1361,17794,28985,2229,0.074923,72583,2.439715,67298,2.262071,0.030710,0.033121,0.013576
1619646,40141,28985,12,0.000403,1177,0.039562,67298,2.262071,0.010195,0.000178,0.004507


In [35]:
item_pairs_fre = item_pairs_fre.sort_values('lift', ascending=False)

In [36]:
item_pairs_fre.head()

,item_A,item_B,freqAB,supportAB,freqA,supportA,freqB,supportB,confidenceAtoB,confidenceBtoA,lift
1417601,30388,27553,159,0.005344,650,0.021848,755,0.025378,0.244615,0.210596,9.639016
561090,29126,36361,306,0.010286,1162,0.039058,839,0.028201,0.263339,0.364720,9.337900
211483,47272,49508,140,0.004706,642,0.021579,729,0.024504,0.218069,0.192044,8.899413
835280,7657,19513,178,0.005983,685,0.023025,898,0.030184,0.259854,0.198218,8.608926
953507,27553,30388,141,0.004739,755,0.025378,650,0.021848,0.186755,0.216923,8.547806


In [37]:
products   = pd.read_csv('./products.csv')
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [38]:
products   = products.rename(columns={'product_id':'item_id', 'product_name':'item_name'})

In [39]:
columns = ['itemA','itemB','freqAB','supportAB','freqA','supportA','freqB','supportB', 
           'confidenceAtoB','confidenceBtoA','lift']
result = (item_pairs_fre
            .merge(products.rename(columns={'item_name': 'itemA'}), left_on='item_A', right_on='item_id')
            .merge(products.rename(columns={'item_name': 'itemB'}), left_on='item_B', right_on='item_id'))

In [42]:
result = result.sort_values('lift', ascending=False)

In [43]:
result = result[columns]

In [46]:
result_to_save = result.head(100)

In [47]:
result_to_save.to_csv("association_result.csv")

In [44]:
result.head(10)

,itemA,itemB,freqAB,supportAB,freqA,supportA,freqB,supportB,confidenceAtoB,confidenceBtoA,lift
0,Organic Forest Berry Cream On Top Whole Milk Y...,Organic Blueberry Cream On Top Whole Milk Yogurt,159,0.005344,650,0.021848,755,0.025378,0.244615,0.210596,9.639016
6474656,Organic Strawberry Chia Lowfat 2% Cottage Cheese,Organic Cottage Cheese Blueberry Acai Chia,306,0.010286,1162,0.039058,839,0.028201,0.263339,0.364720,9.337900
11097625,Antioxidant Infusions Beverage Malawi Mango,Antioxidant Infusions Costa Rica Clementine,140,0.004706,642,0.021579,729,0.024504,0.218069,0.192044,8.899413
13143041,Hair Shampoos,Moroccan Argan Oil + Argan Stem Cell Triple Mo...,178,0.005983,685,0.023025,898,0.030184,0.259854,0.198218,8.608926
14024777,Organic Blueberry Cream On Top Whole Milk Yogurt,Organic Forest Berry Cream On Top Whole Milk Y...,141,0.004739,755,0.025378,650,0.021848,0.186755,0.216923,8.547806
1,Cream on Top Strawberry Organic Yogurt,Organic Blueberry Cream On Top Whole Milk Yogurt,228,0.007664,1110,0.037310,755,0.025378,0.205405,0.301987,8.093955
15171197,Antioxidant Infusions Costa Rica Clementine,Antioxidant Infusions Beverage Malawi Mango,125,0.004202,729,0.024504,642,0.021579,0.171468,0.194704,7.945904
66668,"Yogurt, Sheep Milk, Strawberry","Yogurt, Sheep Milk, Blackberry",238,0.008000,974,0.032739,964,0.032403,0.244353,0.246888,7.541137
7814832,O'Soy Fruit on the Bottom Blueberry Organic So...,O'Soy Fruit on the Bottom Strawberry Organic S...,233,0.007832,900,0.030251,1031,0.034655,0.258889,0.225994,7.470516
7516797,Peach & Mango Soy Yogurt,Blueberry Soy Yogurt,252,0.008470,1156,0.038856,876,0.029445,0.217993,0.287671,7.403456
